In [ ]:
!pip install gradio

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import gradio as gr
import pickle
import numpy as np
import pandas as pd

# 입력을 받을 각 변수에 대한 입력 폼 생성
age_input = gr.inputs.Slider(18, 64, step=1, label="Age👶👵")
gender_input = gr.inputs.Radio(choices=["Female", "Male"], label="Gender👩🧑")
bmi_input = gr.inputs.Number(label="BMI⏲")
children_input = gr.inputs.Number(label="Children👩🏼‍🤝‍👩🏻👩🏾‍🤝‍👩🏾")
smoker_input = gr.inputs.Checkbox(label="Smoker🚬")
region_input = gr.inputs.Radio(choices=["Northeast", "Northwest", "Southeast", "Southwest"], label="Region🌏")

with open("/content/model_RFR.pkl", "rb") as f:
    model = pickle.load(f)

def predict_charges(data):
    age, sex, bmi, children, smoker, region = data

    age = int(age)
    if sex == 'Male' : sex = 'male'
    elif sex == 'Female' : sex = 'female'
    bmi = float(bmi)
    children = int(children)
    if smoker : smoker = 'yes'
    else : smoker = 'no'
    region = region.lower()

    df = pd.DataFrame(columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])
    df.loc[0] = [age, sex, bmi, children, smoker, region]

    df.loc[1, 'sex'] = 'female'
    df.loc[2,'sex'] = 'male'

    df.loc[1, 'smoker'] = 'yes'
    df.loc[2,'smoker'] = 'no'

    df.loc[1, 'region'] = 'southwest'
    df.loc[2,'region'] = 'southeast'
    df.loc[3, 'region'] = 'northwest'
    df.loc[4,'region'] = 'northeast'

    df = pd.get_dummies(df, drop_first=True)

    pred = model.predict([df.loc[0, :].values])
    pred = f'$ {pred[0]:.2f}'

    return pred

# 입력을 받아 예측하는 함수 정의 (여기에서는 임의로 0으로 반환)
def predict(age, sex, bmi, children, smoker, region):

    # 1.0, Male, 2.0, 3.0, True, Northwest
    data = [age, sex, bmi, children, smoker, region]

    prediction = predict_charges(data)
    # 여기에 모델을 사용한 예측 코드를 추가하세요
    # prediction = 0
    return prediction

# 웹 애플리케이션 정의
app = gr.Interface(
    fn=predict,
    inputs=[age_input, gender_input, bmi_input, children_input, smoker_input, region_input],
    outputs="text",
    live=False,
    use_label="False",
    title="Insurance Cost Prediction",
    description="Predict insurance costs based on input variables."
)

# # 출력 필드 추가 (기존 출력 필드와 동일한 값을 출력)
# output_field = gr.outputs.Text(label="Insurance Cost")
# app.add_output(output_field)

# 텍스트 상자와 제출 버튼 추가
submit_btn = gr.Button(value='submit')

# 애플리케이션 실행
app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b9a5ba53eac0efc2f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
